In [ ]:
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import pickle
from scipy import stats

#import netcdf fmc dataset

path_fmc = '/g/data/oe9/project/to_hot_in_here/fmc_upscale/fmc5km_2002_2014.nc'
fmc = Dataset(path_fmc, 'r')

# for item in fmc.variables.keys():
#     print('Variable: \t', item)
#     print('Dimensions: \t', fmc[item].dimensions)
#     print('Shape:    \t', fmc[item].shape, '\n')



layer, smtime = 'sd', []

for year in range(2005,2019):
    
    path_sm = '/g/data/oe9/project/to_hot_in_here/soil_moisture/'+layer+'_pct_'+ str('%i' % year) + '_Actual_day.nc'
    sm = Dataset(path_sm, 'r')
    smtime_aux = sm['time'][:]

    #loop through time to append to our (empty) time/data lists
    for i, t in enumerate(smtime_aux):
        smtime.append(int(t))
    
smtime = np.array(smtime, dtype = int)

fmctime = list(map(int, fmc['time'][:]))
fmctime = np.array(fmctime, dtype = int)

smi, fmci = [], []
for i in range(fmctime.size):
    x = np.where(smtime == fmctime[i])[0]
    if (x.size != 0):
        smi.append(x[0])
        fmci.append(i)
        
smi = np.array(smi, dtype=int)
fmci = np.array(fmci, dtype=int)

# create empty array for storing r (pearson) values
r = np.zeros((681, 841)) * np.nan

for n1 in range(7):

    n1start = n1*100
    n1end = min(n1start+100, 681)
    n1size = n1end - n1start

    for n2 in range(9):
        
        n2start = n2*100
        n2end = min(n2start+100, 841)
        n2size = n2end - n2start
        
        #read AWRA soil moisture from 2005 to 2014
        smdata = np.zeros((smtime.size, n1size, n2size))

        for year in range(2005,2015):
            path_sm = '/g/data/oe9/project/to_hot_in_here/soil_moisture/'+layer+'_pct_'+ str('%i' % year) + '_Actual_day.nc'
            sm = Dataset(path_sm, 'r')
            timeyear = sm['time'][:]
            datayear = sm[layer+'_pct'][:, n1start:n1end, n2start:n2end]

            #loop through time to append to our (empty) time/data lists
            t0 = np.where(smtime == timeyear[0])[0][0]
            tn = np.where(smtime == timeyear[-1])[0][0]
#             print('year =', year)
#             print('n1size =', n1size, ';   n1start =', n1start, ';   n1end =', n1end)
#             print('n2size =', n2size, ';   n2start =', n2start, ';   n2end =', n2end)
#             aux = smdata[t0:tn+1, 0:n1size, 0:n2size]
#             print('smdata[t0:tn+1, 0:n1size, 0:n2size].shape =', aux.shape)
#             print('datayear.shape =', datayear.shape)
            smdata[t0:tn+1, 0:n1size, 0:n2size] = datayear[:,:,:] # n1start:n1end, n2start:n2end]


        # generating 2D matrix with times series for FMC and SM
        a = fmc['fmc_median'][fmci, n1start:n1end, n2start:n2end]
        b = smdata[smi, :, :]

        aa = a.transpose()
        aa = np.rollaxis(aa, 1)
        aa = np.reshape(aa, (n1size*n2size, fmci.size))

        bb = b.transpose()
        bb = np.rollaxis(bb, 1)
        bb = np.reshape(bb, (n1size*n2size, fmci.size))
        
        p = np.zeros((aa.shape[0])) * np.nan
        for i in range(aa.shape[0]):
            mask = np.isfinite([aa[i], bb[i]]).all(axis=0)
            p[i] = np.corrcoef(aa[i,mask], bb[i,mask])[0, 1]
        p = np.reshape(p, (n1size, n2size))
        
        r[n1start:n1end, n2start:n2end] = p[:, :]
        print(n1, n2)
        
with open('pearson_'+layer+'_fmc_4day.pkl','wb') as f:
    pickle.dump(r, f)

print('ok!')

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import ticker
%matplotlib inline
fig = plt.figure(figsize=(16,12))
plt.contourf(fmc['longitude'][:], fmc['latitude'][:], r*r, levels=20, cmap='seismic_r', vmin=0, vmax=1)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=16)
tick_locator = ticker.MaxNLocator(nbins=21)
cbar.locator = tick_locator
cbar.update_ticks()
plt.show()

In [4]:
# monthly linear correlations

from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import pickle
from scipy import stats

#import netcdf fmc dataset

path_fmc = '/g/data/oe9/project/to_hot_in_here/fmc_upscale/fmc5km_2001_2018.nc'
fmc = Dataset(path_fmc, 'r')

# for item in fmc.variables.keys():
#     print('Variable: \t', item)
#     print('Dimensions: \t', fmc[item].dimensions)
#     print('Shape:    \t', fmc[item].shape, '\n')



#Uncoment the block below to store monthly-average variables (fmc and sm)
layer, smtime = 'sd', []
nyears = 2019 - 2005
nmonths = nyears * 12
# vm1 = np.zeros((nmonths, 681, 841)) * np.nan
vm2 = np.zeros((nmonths, 681, 841)) * np.nan

for year in range(2005,2019):
    
    path_sm = '/g/data/oe9/project/to_hot_in_here/soil_moisture/'+layer+'_pct_'+ str('%i' % year) + '_Actual_day.nc'
    sm = Dataset(path_sm, 'r')
    smtime_aux = sm['time'][:]

    #loop through time to append to our (empty) time/data lists
    for i, t in enumerate(smtime_aux):
        smtime.append(int(t))
    
smtime = np.array(smtime, dtype = int)

fmctime = list(map(int, fmc['time'][:]))
fmctime = np.array(fmctime, dtype = int)

# smi, fmci = [], []
# for i in range(fmctime.size):
#     x = np.where(smtime == fmctime[i])[0]
#     if (x.size != 0):
#         smi.append(x[0])
#         fmci.append(i)
        
# smi = np.array(smi, dtype=int)
# fmci = np.array(fmci, dtype=int)

# create empty array for storing r (pearson) values
r = np.zeros((681, 841)) * np.nan
nmonths = (2019-2005)*12

for n1 in range(7):

    n1start = n1*100
    n1end = min(n1start+100, 681)
    n1size = n1end - n1start

    for n2 in range(9):
        
        n2start = n2*100
        n2end = min(n2start+100, 841)
        n2size = n2end - n2start
        
        #read AWRA soil moisture from 2005 to 2014
        smmonths = np.zeros((nmonths, n1size, n2size))
        fmcmonths = np.zeros((nmonths, n1size, n2size))
        m, months = 0, []
        
        for year in range(2005, 2019):
            
            path_sm = '/g/data/oe9/project/to_hot_in_here/soil_moisture/'+layer+'_pct_'+ str('%i' % year) + '_Actual_day.nc'
            sm = Dataset(path_sm, 'r')
            timeyear = sm['time'][:]
            datayear = np.ma.filled(sm[layer+'_pct'][:, n1start:n1end, n2start:n2end], np.nan)
#             print(datayear.shape)
            
            for month in range(1, 13):
                months.append(datetime(year, month, 1))
                t0 = datetime(year, month, 1) - datetime(1900, 1, 1)
                t0 = t0.days
                tn = datetime(year, month, 1) + timedelta(days=40) 
                tn = datetime(tn.year, tn.month, 1)- datetime(1900, 1, 1)
                tn = tn.days
                wfmc = np.where((fmctime >= t0) & (fmctime < tn))[0]#returns a tuple with a list
                wsm = np.where((timeyear >= t0) & (timeyear < tn))[0]
                
                #The block below is the very slow way to compute the mean, pixel by pixel
#                 for i in range(n1size):
#                     for j in range(n2size):
#                         v1 = np.nanmean(fmc['fmc_median'][wfmc, n1start+i, n2start+j]) 
#                         #datayear[wsm, i, j] = 15.
#                         #print(datayear[wsm, i, j])
#                         v2 = np.nanmean(datayear[wsm, i, j])
#                         fmcmonths[m, i, j] = v1
#                         smmonths[m, i, j] = v2
                
                #The block below is the smart and fast way to compute the mean matrix-wise
                try:
                    aux = fmc['fmc_median'][wfmc, n1start:n1end, n2start:n2end]
                except:
                    print(year, month, aux.shape)
                    print(wfmc)
                    print(aux)
                    raise IndexError
                
                fmcmonths[m,:,:] = np.nanmean(aux, axis=0)
                smmonths[m,:,:] = np.nanmean(datayear[wsm,:,:], axis=0)
                
                #The two lines below are for store the complete monthly series
#                 vm1[m, n1start:n1end, n2start:n2end] = fmcmonths[m,:,:]
                vm2[m, n1start:n1end, n2start:n2end] = smmonths[m,:,:]
                m += 1
        
        #aa and bb reshape the fmcmonths and smmonths matrices to flatten the lat/lon dimensions only
        aa = fmcmonths.transpose()
        aa = np.rollaxis(aa, 1)
        aa = np.reshape(aa, (n1size*n2size, nmonths))

        bb = smmonths.transpose()
        bb = np.rollaxis(bb, 1)
        bb = np.reshape(bb, (n1size*n2size, nmonths))
        
        p = np.zeros((aa.shape[0])) * np.nan
        for i in range(aa.shape[0]):
            mask = np.isfinite([aa[i], bb[i]]).all(axis=0)
            p[i] = np.corrcoef(aa[i,mask], bb[i,mask])[0, 1]
        p = np.reshape(p, (n1size, n2size))
        
        r[n1start:n1end, n2start:n2end] = p[:, :]
        print(n1, n2)
        
with open('pearson_'+layer+'_fmc_monthly.pkl','wb') as f:
    pickle.dump(r, f)

#The blocks below are to create a pickle file with monthly data, and the array of values of each dimension.
# with open('monthly_fmc_median.pkl', 'wb') as f:
#     aux = {
#         'months': months,
#         'latitude': fmc['latitude'][:],
#         'longitude': fmc['longitude'][:],
#         'fmc_median': vm1
#     }
#     pickle.dump(aux, f)

with open('monthly_sd.pkl', 'wb') as f:
    aux = {
        'months': months,
        'latitude': fmc['latitude'][:],
        'longitude': fmc['longitude'][:],
        'sd': vm2
    }
    pickle.dump(aux, f)


print('ok!')

/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:111: RuntimeWarning: Mean of empty slice
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:112: RuntimeWarning: Mean of empty slice


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
ok!


In [7]:
from matplotlib import pyplot as plt
from matplotlib import ticker
import pickle
from netCDF4 import Dataset
%matplotlib inline

path_fmc = '/g/data/oe9/project/to_hot_in_here/fmc_upscale/fmc5km_2001_2018.nc'
fmc = Dataset(path_fmc, 'r')

plt.style.use('ggplot')

name = 'pearson_s0_fmc_monthly'
with open(name+'.pkl','rb') as f:
    r = pickle.load(f)
    
fig, ax = plt.subplots(figsize=(16,12))
cf = ax.contourf(fmc['longitude'][:], fmc['latitude'][:], r*r, levels=20, cmap='seismic_r', vmin=0, vmax=1)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
cbar = plt.colorbar(cf)
cbar.ax.tick_params(labelsize=20)
tick_locator = ticker.MaxNLocator(nbins=11)
cbar.locator = tick_locator
cbar.update_ticks()
ax.set_title('r$^2$: Upper SM x FMC', fontsize = 24)
ax.set_facecolor('#3b3e42')
fig.tight_layout()

plt.savefig('01'+ name + '.png')
plt.close(fig)